# DataFrames para funciones y sus respectivas funciones.

En esta sección [Data_Funciones], se llevarán a cabo las reducciones pertinentes a cada DataFrame para luego guardarlos en archivos parquet y utilizarlos más tarde con nuestra API. Además, se desarrollarán las funciones que se emplearán en la API.  
En este notebook, encontrarás el título de cada función, la definición de lo que se espera de cada una de ellas, el procesamiento de los DataFrames y, finalmente, la implementación y prueba de cada función. Las funciones estan divididas por lineas blancas al pasar de una a la otra.

### Importamos librerías

Estas librerías nos permiten manipular los datos y almacenarlos.

In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../')

### Carga de Datos

In [2]:
data_review = pd.read_csv('../datasets/australian_reviews_listo.csv')
data_items = pd.read_csv('../datasets/australian_items.csv')
data_output = pd.read_csv('../datasets/output.csv')
df_recommend = pd.read_csv('../datasets/df_recommend.csv')

### PlayTimeGenre

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

#### Procesamiento de datos

Se unió el DataFrame de los juegos (data_output) con el de items que posee las horas jugadas de cada juego. Luego se eliminaron los datos nulos que se generaron en la fusión y aquellas filas que contenían al menos un valor nulo. Esta decisión se tomó para asegurar que los datos restantes fueran confiables. Además, eliminamos otras columnas que no son relevantes para la función.

In [3]:
data_genres = pd.merge(data_output,data_items,on='item_id',how='right')
data_genres = data_genres.drop(columns=['publisher','app_name','title','price','early_access','developer','item_name','user_id','items_count','steam_id'])
data_genres = data_genres.dropna()

Seleccionamos aquellos juegos y sus géneros que tenían más de 0 horas jugadas.

In [4]:
data_genres = data_genres.loc[data_genres['playtime_forever'] > 0]
data_genres

,genres,release_date,item_id,playtime_forever
0,Action,2000,10,6.0
2,Action,2003,30,7.0
8,Action,2010,300,4733.0
9,Action,2004,240,1853.0
10,Action,2005,3830,333.0
...,...,...,...,...
10942337,Adventure,2015,388490,3.0
10942338,Free to Play,2015,388490,3.0
10942339,Casual,2016,521570,4.0
10942340,Free to Play,2016,521570,4.0


Se agruparon los registros por género y fecha de lanzamiento, además se sumaron la cantidad de horas jugadas. Luego se reseteó el índice para que los identificadores estén con un orden lógico.

In [5]:
data_playtime = data_genres.groupby(['genres','release_date'])['playtime_forever'].sum().reset_index()
data_playtime

,genres,release_date,playtime_forever
0,Action,1983,3582.0
1,Action,1984,384.0
2,Action,1988,16243.0
3,Action,1989,607.0
4,Action,1990,18787.0
...,...,...,...
348,Web Publishing,2013,335849.0
349,Web Publishing,2014,33732.0
350,Web Publishing,2015,348861.0
351,Web Publishing,2016,136.0


### Exportación de datos

Exportamos el dataframe en formato Parquet, ademas de ser mas liviano es mas eficiente que el formato CSV.

In [6]:
data_playtime.to_parquet('../Data_parquet/data_playtime.parquet')

### Función

La función "PlayTimeGenre" tiene dos parámetros que son: un DataFrame y un género que debe ser de tipo string. Se filtra el DataFrame a través del género, se ordenan los valores de forma descendente y se obtiene solo el primer valor, lo mismo para el año de posteo.

In [7]:
def PlayTimeGenre (dataframe,genero:str):
    data_play = dataframe[dataframe['genres'] == genero]
    data_play = data_play.sort_values(by='playtime_forever',ascending=False).head(1)
    
    date = data_play['release_date'].values[0]
    feedback = (f'Año de lanzamiento con más horas jugadas para Género {genero}: {date}')
    return feedback

Revisión de ejecución de la función.

In [8]:
PlayTimeGenre(data_genres,'Indie')

'Año de lanzamiento con más horas jugadas para Género Indie: 2006'

--------------------------------------------------------------------------------------------------------------------------

### UserForGenre

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

#### Procesamiento de datos

Se unificaron dos DataFrames: "data_output" y "data_items", y se eliminaron las columnas que no eran necesarias.

In [9]:
data_user_genre = pd.merge(data_output,data_items)
data_user_genre = data_user_genre.drop(columns=['publisher','app_name','title','price','early_access','developer','item_name','items_count','steam_id'])
data_user_genre

,genres,release_date,item_id,playtime_forever,user_id
0,Action,1997,282010,5.0,UTNerd24
1,Action,1997,282010,0.0,I_DID_911_JUST_SAYING
2,Action,1997,282010,0.0,76561197962104795
3,Action,1997,282010,0.0,r3ap3r78
4,Action,1997,282010,13.0,saint556
...,...,...,...,...,...
9993942,Action,2004,80,0.0,76561198273508956
9993943,Action,2004,80,0.0,76561198282090798
9993944,Action,2004,80,0.0,943525
9993945,Action,2004,80,9.0,76561198283312749


Se agruparon los datos a través del id del usuario, el género del juego y el año. Se sumaron los valores de las horas acumuladas por usuario. Luego, se reseteó el índice para que nuestro nuevo DataFrame "data_user_genre" quede listo para ser consumido por nuestra API.

In [10]:
data_user_genre = data_user_genre.groupby(['user_id','genres','release_date'])['playtime_forever'].sum().reset_index()
data_user_genre

,user_id,genres,release_date,playtime_forever
0,--000--,Action,2009,5329.0
1,--000--,Action,2010,22.0
2,--000--,Action,2011,6522.0
3,--000--,Action,2012,109346.0
4,--000--,Action,2013,363.0
...,...,...,...,...
3495277,zzzmidmiss,Strategy,2009,0.0
3495278,zzzmidmiss,Strategy,2010,10.0
3495279,zzzmidmiss,Strategy,2011,69.0
3495280,zzzmidmiss,Strategy,2012,923.0


Los registros que tenían un valor de 0 en la columna "playtime_forever" se borraron por dos motivos: alivianar la cantidad de registros en el DataFrame y porque no tenían un valor significativo en el objetivo de nuestra función.

In [11]:
data_user_genre = data_user_genre.loc[data_user_genre['playtime_forever']>0]
data_user_genre

,user_id,genres,release_date,playtime_forever
0,--000--,Action,2009,5329.0
1,--000--,Action,2010,22.0
2,--000--,Action,2011,6522.0
3,--000--,Action,2012,109346.0
4,--000--,Action,2013,363.0
...,...,...,...,...
3495275,zzzmidmiss,Sports,2010,194.0
3495276,zzzmidmiss,Sports,2014,16.0
3495278,zzzmidmiss,Strategy,2010,10.0
3495279,zzzmidmiss,Strategy,2011,69.0


### Exportación de datos

Exportamos el dataframe en formato Parquet, ademas de ser mas liviano es mas eficiente que el formato CSV.

In [12]:
data_user_genre.to_parquet('../Data_parquet/data_user_genre.parquet')

### Función

En la función que recibe dos parámetros, un DataFrame y un género de juegos que debe ser string, se filtró nuestro DataFrame "data_user_genre" por el género que se le ingrese. Luego, se ordenaron los valores de manera descendente y se extrajo la primera posición de esos valores. Después, se extrajeron todos los datos: horas de juego, nombre de usuario, años en los que jugó y las horas por cada uno de esos años. Esto último se logró mediante el agrupamiento por año y la suma de las horas jugadas. Por último, se guardaron los valores en un diccionario.

In [13]:
def UserForGenre(data_user_genre,genero:str):

    data_genre = data_user_genre[data_user_genre['genres'] == genero]
    data_usuario = data_genre.sort_values(by='playtime_forever',ascending=False).iloc[0]
    hours = data_usuario.at['playtime_forever']
    data_user =data_usuario.values[0]
    anio = data_usuario.at['release_date']
    horas_anuales_usuario = data_genre.groupby('release_date')['playtime_forever'].sum().reset_index()
    horas_anuales = horas_anuales_usuario.to_dict(orient='records')
    write = (f'Usuario con más horas jugadas para el género {genero} es: {data_user}, en el año {anio}: {hours} jugadas, además tenemos las horas jugadas por año del usuario {data_user}: {horas_anuales}')
    return write              

Revisión de ejecución de la función.

In [14]:
UserForGenre(data_user_genre,'Sports')

"Usuario con más horas jugadas para el género Sports es: 76561198019112245, en el año 2014: 182988.0 jugadas, además tenemos las horas jugadas por año del usuario 76561198019112245: [{'release_date': '1995', 'playtime_forever': 31220.0}, {'release_date': '2005', 'playtime_forever': 69758.0}, {'release_date': '2006', 'playtime_forever': 11211.0}, {'release_date': '2007', 'playtime_forever': 1308.0}, {'release_date': '2008', 'playtime_forever': 98946.0}, {'release_date': '2009', 'playtime_forever': 250785.0}, {'release_date': '2010', 'playtime_forever': 2397026.0}, {'release_date': '2011', 'playtime_forever': 930765.0}, {'release_date': '2012', 'playtime_forever': 3608452.0}, {'release_date': '2013', 'playtime_forever': 6257701.0}, {'release_date': '2014', 'playtime_forever': 5604627.0}, {'release_date': '2015', 'playtime_forever': 41795973.0}, {'release_date': '2016', 'playtime_forever': 4763913.0}, {'release_date': '2017', 'playtime_forever': 22876.0}, {'release_date': 'No data', 'play

--------------------------------------------------------------------------------------------------------------------------

### UsersRecommend

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

#### Procesamiento de datos

Se unificaron los DataFrames "data_output" y "data_review" para dar lugar a "data_recommend", y se eliminaron las columnas que no serán necesarias para nuestra función.

In [15]:
data_recommend = pd.merge(data_output,data_review,on='item_id')
data_recommend = data_recommend.drop(columns=['publisher','genres','title','release_date','price','early_access','developer','user_url'])
data_recommend

,app_name,item_id,user_id,posted,recommend,sentiment_analysis
0,Carmageddon Max Pack,282010,InstigatorAU,2014,True,1
1,Carmageddon Max Pack,282010,InstigatorAU,2014,True,1
2,Carmageddon Max Pack,282010,InstigatorAU,2014,True,1
3,Half-Life,70,EizanAratoFujimaki,2015,True,1
4,Half-Life,70,76561198020928326,2014,True,1
...,...,...,...,...,...,...
126184,Counter-Strike: Condition Zero,80,76561198023508728,2014,False,1
126185,Counter-Strike: Condition Zero,80,Lone_walker,2013,True,2
126186,Counter-Strike: Condition Zero,80,virex4,2011,True,2
126187,Counter-Strike: Condition Zero,80,KILLERamateur,2014,True,2


Se realiza un filtrado del DataFrame "data_recommend" con las condiciones de que el juego sea recomendado y además tenga un comentario que puede ser positivo tanto como neutral.

In [16]:
data_recommend = data_recommend[(data_recommend['recommend'] == True) & (data_recommend['sentiment_analysis'].isin([1,2]))]
data_recommend

,app_name,item_id,user_id,posted,recommend,sentiment_analysis
0,Carmageddon Max Pack,282010,InstigatorAU,2014,True,1
1,Carmageddon Max Pack,282010,InstigatorAU,2014,True,1
2,Carmageddon Max Pack,282010,InstigatorAU,2014,True,1
3,Half-Life,70,EizanAratoFujimaki,2015,True,1
4,Half-Life,70,76561198020928326,2014,True,1
...,...,...,...,...,...,...
126180,Counter-Strike: Condition Zero,80,bindisposerer,2015,True,1
126181,Counter-Strike: Condition Zero,80,174gamecuman700kngkakak,2014,True,1
126185,Counter-Strike: Condition Zero,80,Lone_walker,2013,True,2
126186,Counter-Strike: Condition Zero,80,virex4,2011,True,2


Se realiza una agrupación de los registros a través de su nombre y el año en el que fue recomendado y comentado por los usuarios.

In [17]:
data_recommend = data_recommend.groupby(['app_name','posted'])['recommend'].sum().reset_index()
data_recommend

,app_name,posted,recommend
0,//N.P.P.D. RUSH//- The milk of Ultraviolet,2015,2
1,0RBITALIS,2014,2
2,"10,000,000",2014,8
3,"10,000,000",2015,4
4,100% Orange Juice,2014,12
...,...,...,...
4602,theHunter Classic,2014,100
4603,theHunter Classic,2015,65
4604,theHunter: Primal,2014,12
4605,theHunter: Primal,2015,24


### Exportación de datos

Exportamos el dataframe en formato Parquet, ademas de ser mas liviano es mas eficiente que el formato CSV.

In [18]:
data_recommend.to_parquet('../Data_parquet/data_recommend.parquet')

### Función

La función recibe dos parámetros, un DataFrame y el año que se desee ingresar. Se realiza un filtro para el año ingresado y luego se procede a ordenar de manera descendente los valores de la columna "recommend". De la misma se toman los tres primeros valores y el nombre de esos juegos.

In [19]:
def UsersRecommend(dataframe,anio:int):
    datarecommend = dataframe[dataframe['posted'] == anio]
    top3_games = datarecommend.sort_values(by= 'recommend',ascending= False).iloc[0:3]
    names = top3_games['app_name'].tolist()
    anio = top3_games['posted'].tolist()
    write = f'Puesto 1:{names[0]} , Puesto 2:{names[1]} , Puesto 3:{names[2]}'
    return write

Revisión de ejecución de la función.

In [20]:
UsersRecommend(data_recommend,2015)

'Puesto 1:Counter-Strike: Global Offensive , Puesto 2:Unturned , Puesto 3:Team Fortress 2'

--------------------------------------------------------------------------------------------------------------------------

### UsersWorstDeveloper

def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

#### Procesamiento de datos

Se verificaron dos igualdades: si el juego era recomendado o no y si el comentario era negativo. Se agruparon los registros de esas igualdades a través de su identificador (item_id) y el año de posteo, además de resetear el índice. Luego se unió el DataFrame "count_worst_reviews" con el DataFrame "data_output". Se quitaron columnas innecesarias para la función.

In [21]:
worst_reviews = data_review[(data_review['recommend'] == False) & (data_review['sentiment_analysis'] == 0)]
count_worst_reviews = worst_reviews.groupby(['item_id','posted']).size().reset_index(name='cbad_reviews')
df_users = pd.merge(count_worst_reviews,data_output)
df_users = df_users.drop(columns=['early_access','price','release_date','app_name','title','genres','publisher'])
df_users


,item_id,posted,cbad_reviews,developer
0,10,2015,1,Valve
1,20,2015,1,Valve
2,80,2014,1,Valve
3,220,2014,1,Valve
4,220,2015,1,Valve
...,...,...,...,...
1737,501760,2014,1,Kiddy
1738,501760,2014,1,Kiddy
1739,502550,2014,1,Freakinware Limited
1740,502550,2014,1,Freakinware Limited


### Exportación de datos

Exportamos el dataframe en formato Parquet, ademas de ser mas liviano es mas eficiente que el formato CSV.

In [22]:
df_users.to_parquet('../Data_parquet/data_users.parquet')

### Función

La función "UsersWorstDeveloper" recibe dos parámetros: un DataFrame y el año que debe ser de tipo entero. Se instancia un nuevo DataFrame filtrado con solo el año que se pide. Luego, se seleccionan solo los 3 primeros desarrolladores que nos da el contador de valores, y se hace un ciclo for para devolver en qué posición y qué desarrolladores son los 3 peores.

In [23]:
def UsersWorstDeveloper(dataframe,anio:int):

    dfworst = dataframe[(dataframe['posted'] == anio)]

    count_worst_dev = dfworst['developer'].value_counts()

    top_3_worst = count_worst_dev.head(3)

    top_3 = []

    for position,(developer,_) in enumerate(top_3_worst.items()):
        top_3.append({f'Puesto {position+1}:':developer})

    return top_3

Revisión de ejecución de la función.

In [24]:
UsersWorstDeveloper(df_users,2015)

[{'Puesto 1:': 'Cherry Pop Games'},
 {'Puesto 2:': 'ProjectorGames'},
 {'Puesto 3:': 'Valve'}]

--------------------------------------------------------------------------------------------------------------------------

### Sentiment_analysis

def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

#### Procesamiento de datos

Se unieron dos DataFrames, "data_output" y "data_review", para luego dar lugar a un nuevo DataFrame llamado "data_sentiment". Posteriormente, se eliminaron las columnas que no estábamos utilizando o que no serían necesarias para las funciones de la API.

In [25]:
data_sentiment = pd.merge(data_output,data_review, on='item_id')
data_sentiment = data_sentiment.drop(columns=['recommend','publisher','genres','app_name','title','release_date','price','early_access','user_id','user_url','posted'])
data_sentiment

,item_id,developer,sentiment_analysis
0,282010,Stainless Games Ltd,1
1,282010,Stainless Games Ltd,1
2,282010,Stainless Games Ltd,1
3,70,Valve,1
4,70,Valve,1
...,...,...,...
126184,80,Valve,1
126185,80,Valve,2
126186,80,Valve,2
126187,80,Valve,2


Se agrupó por desarrolladora de juegos y por análisis de sentimiento. Con la función "size", se sumaron los elementos de cada valor posible (0, 1 y 2) para las columnas de análisis de sentimiento. Para una mejor visualización, se utilizó la función "unstack", ya que "size" convierte los datos a un formato menos agradable visualmente. Por último, y para asegurar que ninguna desarrolladora tenga valor cero en las tres columnas, se utilizó un filtro para ello.

In [26]:
data_sentiment = data_sentiment.groupby(['developer','sentiment_analysis']).size().unstack(fill_value=0)
drop = data_sentiment.loc[(data_sentiment == 0).all(axis=1)] 
data_sentiment = data_sentiment.drop(index=drop.index)
data_sentiment

sentiment_analysis,0,1,2
developer,,,
07th Expansion,0,3,0
"10th Art Studio,Adventure Productions",0,4,0
10tons Ltd,0,3,0
11 bit studios,24,150,15
14° East,1,1,0
...,...,...,...
xXarabongXx,0,6,0
△○□× (Miwashiba),0,6,4
"インレ,Inre",0,8,2


### Exportación de datos

Exportamos el dataframe en formato Parquet, ademas de ser mas liviano es mas eficiente que el formato CSV.

In [27]:
data_sentiment.to_parquet('../Data_parquet/data_sentiment.parquet')

### Función

La función creada recibe dos parámetros: un DataFrame y el nombre de la desarrolladora de juegos, que debe ser de tipo string. Se realiza una búsqueda con la función "loc" a través del índice, que sería el nombre de la desarrolladora, y por el nombre de las columnas. Luego, se convierte a una lista y se imprime dentro de un diccionario.

In [28]:
def sentiment_analysis( dataframe,desarrolladora:str ):
    developer2= dataframe.loc[desarrolladora,[0,1,2]].to_list()
    write = {desarrolladora:developer2}
    return write

Revisión de ejecución de la función.

In [29]:
sentiment_analysis(data_sentiment,'07th Expansion')

{'07th Expansion': [0, 3, 0]}

### Conclusión

Terminadas las transformaciones específicas para cada dataframe y con su función correctamente empleada debajo, se procede a ir a la última etapa que es consultar las funciones en el archivo [main.py](../main.py), ejecutarlas con FastApi de manera local y, por último, desplegarlas en Render desde la web. El archivo [requirements.txt](../requirements.txt) posee todas las bibliotecas con sus versiones específicas que se utilizaron para el proceso de este proyecto.
Además, se puede acceder al video con la explicación y resumen del proyecto.
